In [ ]:
x=["-1"]

In [1]:
import scipy.stats as spst
import scipy
from scipy import optimize
import seaborn as sns
import numpy as np
import pandas as pd
import qrbook_funcs as qf
from scipy import stats
from tabulate import tabulate
from arch import arch_model
import math
import statistics 
import matplotlib.pyplot as plt
%matplotlib inline

#### Q1 a) What's OAS

In [2]:
def rate_curve(t):
    return (6-5*math.exp(-t/30))/100

In [3]:
def bondPrice(t,OAS):
    time_array=np.arange(1,t+1)
    coupon_list_dis=[]
    for time in time_array:
        r=(6-5*math.exp(-time/30))/100
        coupon_dis=(100*r+3)*math.exp(-time*(r+OAS/100))
        coupon_list_dis.append(coupon_dis)
    
    r_end=(6-5*math.exp(-t/30))/100
    coupon_list_dis.append(100*math.exp(-1*t*(r_end+OAS/100)))
    return sum(coupon_list_dis)

In [4]:
bondPrice(30,3.1415926)

82.45374686877977

In [5]:
current_price=82.4537464147

In [6]:
def bondPriceDiff(OAS):
    return abs(bondPrice(30,OAS)-current_price)

In [7]:
x=scipy.optimize.minimize_scalar(bondPriceDiff)
OAS=x.x
OAS

3.141592657730998

#### The OAS is 3.141592657730998%

#### Q1 b) What's effective duration and spread duration

In [8]:
def bondPrice2(T,OAS,delta):
    time_array=np.arange(1,T+1)
    coupon_list_dis=[]
    for time in time_array:
        r=(6-5*math.exp(-time/30))/100+delta
        coupon_dis=(100*r+3)*math.exp(-time*(r+OAS/100))
        coupon_list_dis.append(coupon_dis)
    
    r_end=(6-5*math.exp(-T/30))/100
    coupon_list_dis.append(100*math.exp(-1*T*(r_end+OAS/100+delta)))
    return sum(coupon_list_dis)

In [9]:
delta=0.0001
price_increase_delta=bondPrice2(30,OAS,delta)
price_decrease_delta=bondPrice2(30,OAS,-delta)
effective_DR=(price_decrease_delta-price_increase_delta)/(2*delta*current_price)
print("Effective duration rate is: ", effective_DR)

Effective duration rate is:  -1.9303379385854385


In [10]:
spread=1
price_increase_sp=bondPrice2(30,OAS-spread,delta)
price_decrease_sp=bondPrice2(30,OAS+spread,-delta)
sp_DR=(price_decrease_sp-price_increase_sp)/(2*spread*current_price)
print("Option-adjusted spread duration rate is: ", sp_DR)

Option-adjusted spread duration rate is:  -0.14300003261835548


#### Q1 c) Why effective rate duration is not close to 0

#### Because the rate curve in our question is not flat. Also the coupon in the nominator is dependent on the rate curve which is not flat

#### Q2 (a) Find the price

In [11]:
def bondPrice_default(T, coupon, p_def, R, OAS):
    
    # time period
    time_array=np.arange(1,T+1)
    
    # first term
    first_term_sum=0    
    for time in time_array:
        first_term_sum=first_term_sum+((1-p_def)**time)/((1+(rate_curve(time)+OAS))**time)
    
    first_term=first_term_sum*(R*p_def/(1-p_def))
    
    # second term
    second_term=coupon*first_term_sum
    
    # third term
    last_term=100*((1-p_def)**T/((1+(rate_curve(T)+OAS))**T))
    
    # result
    res=first_term+second_term+last_term
        
    return res

In [12]:
bondPrice_default(30,2,0.02,50,0)

61.560233352571075

In [13]:
def bondPrice_default2(T,coupon, p_def, R):
    # time period
    time_array=np.arange(1,T+1)
    
    # first term
    first_term_sum=0
    for time in time_array:
        v=(1+rate_curve(time))/(1-p_def)-1
        first_term_sum=first_term_sum+1/((1+v)**time)
    first_term=(R*p_def/(1-p_def)+coupon)*first_term_sum
    
    v_last=(1+rate_curve(T))/(1-p_def)-1
    last_term=100*(1/((1+v_last)**T))   
        
    return first_term+last_term

In [14]:
bondPrice_default(30,2,0.02,50,0)==bondPrice_default2(30,2,0.02,50)

True

In [15]:
price_def=bondPrice_default(30,2,0.02,50,0)
print("The price with default is: ", price_def)

The price with default is:  61.560233352571075


#### Q2 (b) Then assume the credit risk in the bond is accounted for by a spread as in (10.2a)

In [16]:
def bondPriceDiffDef(spread):
    return abs(bondPrice(30,spread)-price_def)

In [17]:
scipy.optimize.minimize_scalar(bondPriceDiffDef)

     fun: 2.5183309304566137e-07
    nfev: 31
     nit: 25
 success: True
       x: 5.359786685454795

#### The spread is 5.359786685454795%

### The spread now is higher than the one in the Q1 because now it's possible that the bond will default. So the spread is higher to compenstate the risk

#### Q3 Compute the risk-neutral default probability

In [18]:
def rate_curve_linear(t):
    return (1+(6-1)/10*t)/100

In [19]:
def bondPriceDefaultFree(T):
    time_array=np.arange(1,T+1)
    
    # compute the discounted cash flow
    price_sum=0
    for time in time_array:
        r=rate_curve_linear(time)
        price_sum=price_sum+2/((1+r)**time)
        #print(r,time,1/((1+r)**time))
    
    r_end=rate_curve_linear(T)
    last=100/((1+r_end)**T)
    #print(last)
    #print(price_sum,last)
    price=price_sum+last
    return price

In [20]:
price_default_free=bondPriceDefaultFree(10)
price_default_free

71.80639062243445

In [21]:
def bondPriceWithDefault(p_def):
    
    # time period
    time_array=np.arange(1,11)
    
    # first term
    first_term_sum=0    
    for time in time_array:
        first_term_sum=first_term_sum+((1-p_def)**time)/((1+(rate_curve_linear(time)))**time)
        #print(time,((1-p_def)**time)/((1+(rate_curve_linear(time)))**time))
    
    first_term=first_term_sum*(40*p_def/(1-p_def))
    
    # second term
    second_term=2*first_term_sum
    
    # third term
    last_term=100*((1-p_def)**10)/(1+(rate_curve_linear(10)))**10
    
    #print(first_term, second_term, last_term)
    
    # result
    res=first_term+second_term+last_term
        
    return res

In [22]:
price_no_default=bondPriceWithDefault(0)
price_no_default

71.80639062243445

In [23]:
price_2_less=price_no_default*(1-0.02)
price_5_less=price_no_default*(1-0.05)
price_10_less=price_no_default*(1-0.1)

In [24]:
def bondPriceDiffDefProbability2(P_def):
    return abs(bondPriceWithDefault(P_def)-price_2_less)

def bondPriceDiffDefProbability5(P_def):
    return abs(bondPriceWithDefault(P_def)-price_5_less)

def bondPriceDiffDefProbability10(P_def):
    return abs(bondPriceWithDefault(P_def)-price_10_less)

In [25]:
res2=scipy.optimize.minimize_scalar(bondPriceDiffDefProbability2,bounds=(0,1),method='bounded')
res5=scipy.optimize.minimize_scalar(bondPriceDiffDefProbability5,bounds=(0,1),method='bounded')
res10=scipy.optimize.minimize_scalar(bondPriceDiffDefProbability10,bounds=(0,1),method='bounded')

In [26]:
print("Risk-neutral probability of default when the price of bond is 2% less than default-free bond is: ",res2.x)
print("Risk-neutral probability of default when the price of bond is 5% less than default-free bond is: ",res5.x)
print("Risk-neutral probability of default when the price of bond is 10% less than default-free bond is: ",res10.x)

Risk-neutral probability of default when the price of bond is 2% less than default-free bond is:  0.004600277354546295
Risk-neutral probability of default when the price of bond is 5% less than default-free bond is:  0.011884622653573885
Risk-neutral probability of default when the price of bond is 10% less than default-free bond is:  0.0252424875285637


In [27]:
# checking
# price_2_less,price_5_less,price_10_less

In [28]:
# checking
# bondPriceWithDefault(res2.x),bondPriceWithDefault(res5.x),bondPriceWithDefault(res10.x)